```
--- Day 21: Scrambled Letters and Hash ---

The computer system you're breaking into uses a weird scrambling function to store its passwords. It shouldn't be much trouble to create your own scrambled password so you can add it to the system; you just have to implement the scrambler.

The scrambling function is a series of operations (the exact list is provided in your puzzle input). Starting with the password to be scrambled, apply each operation in succession to the string. The individual operations behave as follows:

swap position X with position Y means that the letters at indexes X and Y (counting from 0) should be swapped.
swap letter X with letter Y means that the letters X and Y should be swapped (regardless of where they appear in the string).
rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
rotate based on position of letter X means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
reverse positions X through Y means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
move position X to position Y means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.
For example, suppose you start with abcde and perform the following operations:

swap position 4 with position 0 swaps the first and last letters, producing the input for the next step, ebcda.
swap letter d with letter b swaps the positions of d and b: edcba.
reverse positions 0 through 4 causes the entire string to be reversed, producing abcde.
rotate left 1 step shifts all letters left one position, causing the first letter to wrap to the end of the string: bcdea.
move position 1 to position 4 removes the letter at position 1 (c), then inserts it at position 4 (the end of the string): bdeac.
move position 3 to position 0 removes the letter at position 3 (a), then inserts it at position 0 (the front of the string): abdec.
rotate based on position of letter b finds the index of letter b (1), then rotates the string right once plus a number of times equal to that index (2): ecabd.
rotate based on position of letter d finds the index of letter d (4), then rotates the string right once, plus a number of times equal to that index, plus an additional time because the index was at least 4, for a total of 6 right rotations: decab.
After these steps, the resulting scrambled password is decab.

Now, you just need to generate a new scrambled password and you can access the system. Given the list of scrambling operations in your puzzle input, what is the result of scrambling abcdefgh?

```

In [1]:
import re

RE_SWAP_POS = re.compile("swap position ([0-9]+) with position ([0-9]+)")
RE_SWAP_LETTER = re.compile("swap letter ([a-z]) with letter ([a-z])")
RE_ROTATE_LEFT = re.compile("rotate left ([0-9]+) step[s]*")
RE_ROTATE_RIGHT = re.compile("rotate right ([0-9]+) step[s]*")
RE_ROTATE_BASED = re.compile("rotate based on position of letter ([a-z])")
RE_REVERSE_POS = re.compile("reverse positions ([0-9]+) through ([0-9]+)")
RE_MOVE_POS = re.compile("move position ([0-9]+) to position ([0-9]+)")

class Scrambler:
    
    def __init__(self, word):
        self.word = word
    
    def swap_position(self, x, y):        
        a, b = self.word[x], self.word[y]
        if x < y:
            self.word = self.word[:x] + b + self.word[x+1:y] + a + self.word[y+1:]
        else:
            self.word = self.word[:y] + a + self.word[y+1:x] + b + self.word[x+1:]
        
    def swap_letter(self, x, y):
        self.word = self.word.replace(x, '#')
        self.word = self.word.replace(y, '$')
        self.word = self.word.replace('#', y)
        self.word = self.word.replace('$', x)
        
    def rotate_left(self, x):        
        for _ in range(x):
            self.word = self.word[1:] + self.word[0]

    def rotate_right(self, x):
        for _ in range(x):
            self.word = self.word[-1] + self.word[:-1]

    def rotate_based(self, x):
        idx = self.word.index(x)
        self.rotate_right(idx + 1)
        if idx >= 4:
            self.rotate_right(1)
    
    def reverse_position(self, x, y):
        a = min(x, y)
        b = max(x, y)
        self.word = self.word[:a] + ''.join(reversed(self.word[a:b+1])) + self.word[b+1:]
        
    def move_position(self, x, y):  
        if x < y:
            self.word = self.word[:x] + self.word[x+1:y+1] + self.word[x] + self.word[y+1:]
        else:
            self.word = self.word[:y] + self.word[x] + self.word[y:x] + self.word[x+1:]
            
    def process(self, instructions):
        
        for ins in instructions:
            ins = ins.strip()            
            
            m = RE_SWAP_POS.match(ins)
            if m is not None:
                x, y = m.groups()
                self.swap_position(int(x), int(y))
                continue
                
            m = RE_SWAP_LETTER.match(ins)
            if m is not None:
                x, y = m.groups()
                self.swap_letter(x, y)
                continue
                
            m = RE_ROTATE_LEFT.match(ins)
            if m is not None:
                x = m.groups()[0]
                self.rotate_left(int(x))
                continue
        
            m = RE_ROTATE_RIGHT.match(ins)
            if m is not None:
                x = m.groups()[0]
                self.rotate_right(int(x))
                continue
                
            m = RE_ROTATE_BASED.match(ins)
            if m is not None:
                x = m.groups()[0]
                self.rotate_based(x)
                continue
                
            m = RE_REVERSE_POS.match(ins)
            if m is not None:
                x, y = m.groups()
                self.reverse_position(int(x), int(y))
                continue
                
            m = RE_MOVE_POS.match(ins)
            if m is not None:
                x, y = m.groups()
                self.move_position(int(x), int(y))
                continue 
                
        return self.word            

In [2]:
s = Scrambler("abcde")
s.process([
        "swap position 4 with position 0",
        "swap letter d with letter b",
        "reverse positions 0 through 4",
        "rotate left 1 step",
        "move position 1 to position 4",
        "move position 3 to position 0",
        "rotate based on position of letter b",
        "rotate based on position of letter d"        
    ])

'decab'

In [3]:
with open('inputs/day21.txt', 'rt') as fd:
    s = Scrambler("abcdefgh")
    print(s.process(fd))

gbhafcde


```
--- Part Two ---

You scrambled the password correctly, but you discover that you can't actually modify the password file on the system. You'll need to un-scramble one of the existing passwords by reversing the scrambling process.

What is the un-scrambled version of the scrambled password fbgdceah?
```

In [4]:
from itertools import permutations

def decode(value, instructions):
    for p in permutations(value):
        if value == Scrambler("".join(p)).process(instructions):
            yield "".join(p)

In [5]:
list(decode("decab", [
        "swap position 4 with position 0",
        "swap letter d with letter b",
        "reverse positions 0 through 4",
        "rotate left 1 step",
        "move position 1 to position 4",
        "move position 3 to position 0",
        "rotate based on position of letter b",
        "rotate based on position of letter d"        
    ]))

['deabc', 'abcde']

In [6]:
%%time
with open('inputs/day21.txt', 'rt') as fd:
    instructions = fd.readlines()
    print(list(decode("fbgdceah", instructions)))

['bcfaegdh']
CPU times: user 19 s, sys: 8 ms, total: 19 s
Wall time: 19 s
